<a href="https://colab.research.google.com/github/GummyBear-w/aop113b/blob/main/EX02_04_Download_Pokemon_Images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
url = 'https://tw.portal-pokemon.com/play/pokedex/0001'

response = requests.get(url)
response.text

'<!doctype html>\n<html lang="zh-cmn-Hant-TW">\n    <head prefix="og: http://ogp.me/ns# fb: http://ogp.me/ns/fb# website: http://ogp.me/ns/website#">\n        <meta charset="utf-8">\n<meta http-equiv="X-UA-Compatible" content="IE=edge">\n<meta name="viewport" content="width=device-width, initial-scale=1">\n<meta name="csrf-token" content="">\n<title>妙蛙種子 | 寶可夢圖鑑 | The official Pokémon Website in Taiwan</title>\n<meta name="keywords" content="妙蛙種子,寶可夢,寶可夢,Pokemon,寶可夢圖鑑">\n<meta name="description" content="在出生後的一段時間內，牠會吸收背上種子裡儲存著的營養成長。">\n<meta property="og:title" content="妙蛙種子 | 寶可夢圖鑑 | The official Pokémon Website in Taiwan">\n<meta property="og:description" content="在出生後的一段時間內，牠會吸收背上種子裡儲存著的營養成長。">\n<meta property="og:image" content="https://tw.portal-pokemon.com/play/resources/pokedex/img/pm/cf47f9fac4ed3037ff2a8ea83204e32aff8fb5f3.png">\n<meta property="og:type" content="article">\n<meta property="og:site_name" content="The official Pokémon Website in Taiwan">\n<meta name="twitter:ca

In [ ]:
soup = BeautifulSoup(response.text, 'lxml')

In [ ]:
img_url = 'https://tw.portal-pokemon.com' + soup.find('img', class_='pokemon-img__front')['src']
img = requests.get(img_url)
with open('0001.png', 'wb') as f:
    f.write(img.content)

In [ ]:
for i in range(1, 11):
    id = f'{i:04}'
    url = f'https://tw.portal-pokemon.com/play/pokedex/{id}'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')
    img_src = soup.find('img', class_='pokemon-img__front')['src']
    img_url = 'https://tw.portal-pokemon.com' + img_src
    img = requests.get(img_url)
    with open (f'/content/pokemon/{id}.png', 'wb') as f:
        f.write(img.content)

**也許你想知道 我喜歡這些寶可夢ㄡ**

In [ ]:
!pip install gradio -q

In [ ]:
import gradio as gr
import requests
from bs4 import BeautifulSoup

# 指定的寶可夢編號與對應的名稱（這裡可以擴充）
pokemon_dict = {
    "129": "鯉魚王",
    "282": "沙奈朵",
    "700": "仙子伊布",
    "702": "咚咚鼠",
    "704": "黏黏寶",
    "722": "木木梟",
    "763": "甜冷美后",
    "963": "波普海豚",
    "978": "米立龍"
}

# 預設圖片（當抓不到寶可夢圖片時顯示）
DEFAULT_IMAGE_URL = "https://upload.wikimedia.org/wikipedia/commons/9/98/International_Pokémon_logo.svg"

# 爬取寶可夢資料
def get_pokemon_info(pokemon_id):
    url = f'https://tw.portal-pokemon.com/play/pokedex/{pokemon_id}'
    headers = {"User-Agent": "Mozilla/5.0"}  # 避免被網站封鎖
    response = requests.get(url, headers=headers)

    if response.status_code != 200:
        return "找不到資料", DEFAULT_IMAGE_URL

    soup = BeautifulSoup(response.text, 'lxml')

    # 取得描述
    description_tag = soup.find('p', class_='pokemon-story__body')
    description = description_tag.text.strip() if description_tag else "無法獲取描述"

    # 取得圖片 URL
    img_tag = soup.find('img', class_='pokemon-img__front')
    img_url = 'https://tw.portal-pokemon.com' + img_tag['src'] if img_tag and img_tag.has_attr('src') else DEFAULT_IMAGE_URL

    # 屬性跟弱點只保留中文字

    types_tag = soup.find_all('div', class_='pokemon-type__type')
    types = ', '.join([t.text.strip() for t in types_tag]) if types_tag else "無法獲取屬性"

    weaknesses_tag = soup.find_all('div', class_='pokemon-weakness__btn')
    weaknesses = ', '.join([w.text.strip() for w in weaknesses_tag]) if weaknesses_tag else "無法獲取弱點"

    return description, img_url, types, weaknesses

# Gradio 介面函數
def show_pokemon(pokemon_name):
    if not pokemon_name:
        return "請選擇寶可夢", DEFAULT_IMAGE_URL

    pokemon_id = next((k for k, v in pokemon_dict.items() if v == pokemon_name), None)

    if not pokemon_id:
        return "無法找到該寶可夢", DEFAULT_IMAGE_URL

    return get_pokemon_info(pokemon_id)

In [ ]:
with gr.Blocks() as demo:
    gr.Markdown("## 🏆 寶可夢資料查詢")

    with gr.Row():
        # 左邊是選擇框
        with gr.Column(scale=1):
            pokemon_name = gr.Dropdown(
                choices=["請選擇寶可夢"] + list(pokemon_dict.values()),  # 第一個選項為提示
                label="選擇寶可夢",
                value="請選擇寶可夢",  # 預設為提示文字
                interactive=True
            )

        # 右邊顯示結果
        with gr.Column(scale=2):
            output_text = gr.Textbox(label="描述", value="請選擇寶可夢以查看資訊")
            output_types = gr.Textbox(label="屬性", value="請選擇寶可夢以查看資訊")
            output_weaknesses = gr.Textbox(label="弱點", value="請選擇寶可夢以查看資訊")
            output_image = gr.Image(label="圖片", value=DEFAULT_IMAGE_URL, type="filepath")


    # 當選擇變更時，自動更新資訊
    pokemon_name.change(fn=show_pokemon, inputs=pokemon_name, outputs=[output_text, output_image, output_types, output_weaknesses])

demo.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4387b7e92d5ccafaca.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
